In [ ]:
import os
import sqlite3
import fitz  # PyMuPDF
from flask import Flask, request, redirect, url_for, render_template, send_from_directory, g
from werkzeug.utils import secure_filename
from yolo_helpers import run_inference # Import our custom YOLO function

# --- Configuration ---
UPLOAD_FOLDER = 'static/uploads'
FACE_FOLDER = 'static/faces'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'pdf'}
DATABASE = 'onboarding.db'
MODEL_PATH = 'runs/detect/train/weights/best.pt' # Path to your trained model

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['FACE_FOLDER'] = FACE_FOLDER
app.config['MODEL_PATH'] = MODEL_PATH

# --- Database Helpers ---

def get_db():
    """Opens a new database connection if one is not already open."""
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
        db.row_factory = sqlite3.Row # Allows accessing columns by name
    return db

@app.teardown_appcontext
def close_connection(exception):
    """Closes the database connection at the end of the request."""
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

def query_db(query, args=(), one=False):
    """Helper function to query the database."""
    cur = get_db().execute(query, args)
    rv = cur.fetchall()
    cur.close()
    return (rv[0] if rv else None) if one else rv

def insert_db(query, args=()):
    """Helper function to insert data and return the new row's ID."""
    db = get_db()
    cur = db.execute(query, args)
    db.commit()
    new_id = cur.lastrowid
    cur.close()
    return new_id

# --- File Helpers ---

def allowed_file(filename):
    """Checks if the file extension is allowed."""
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def convert_pdf_to_image(pdf_path, output_image_path):
    """Converts the first page of a PDF to a PNG image using PyMuPDF."""
    try:
        doc = fitz.open(pdf_path)
        page = doc.load_page(0)  # Load the first page
        
        # Render page to a pixmap (image) at 300 DPI
        pix = page.get_pixmap(dpi=300) 
        
        pix.save(output_image_path)
        doc.close()
        print(f"Successfully converted {pdf_path} to {output_image_path}")
        return output_image_path
    except Exception as e:
        print(f"Error converting PDF: {e}")
        return None

# --- Flask Routes ---

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url) # No file part
        
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url) # No selected file

        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            original_filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(original_filepath)
            
            # --- Processing Pipeline ---
            image_to_process = original_filepath
            
            # 1. Convert PDF to Image if necessary
            if filename.lower().endswith('.pdf'):
                print(f"Detected PDF: {filename}. Converting...")
                png_filename = f"{os.path.splitext(filename)[0]}.png"
                image_to_process = os.path.join(app.config['UPLOAD_FOLDER'], png_filename)
                
                if not convert_pdf_to_image(original_filepath, image_to_process):
                    # Handle conversion error
                    return "Error converting PDF", 500
            
            # 2. Run YOLO Inference
            # This function is defined in yolo_helpers.py
            # It returns the extracted text data and the path to the cropped face
            try:
                extracted_data, face_image_path = run_inference(
                    model_path=app.config['MODEL_PATH'],
                    source_image_path=image_to_process,
                    face_output_dir=app.config['FACE_FOLDER']
                )
            except Exception as e:
                print(f"Error during YOLO inference: {e}")
                return f"Error running AI model. Is '{app.config['MODEL_PATH']}' your trained model?", 500

            # 3. Store in SQLite Database
            try:
                customer_id = insert_db(
                    'INSERT INTO customers (name, dob, id_number, face_image_url) VALUES (?, ?, ?, ?)',
                    [
                        extracted_data.get('name', 'N/A'),
                        extracted_data.get('dob', 'N/A'),
                        extracted_data.get('id_number', 'N/A'),
                        face_image_path # Store the *relative* path to the face
                    ]
                )
                print(f"Stored data in DB with ID: {customer_id}")
                
                # 4. Redirect to the generated form
                return redirect(url_for('show_form', customer_id=customer_id))
                
            except Exception as e:
                print(f"Error inserting into database: {e}")
                return "Error saving to database", 500

    # Handle GET request
    return render_template('index.html')


@app.route('/form/<int:customer_id>')
def show_form(customer_id):
    """Displays the generated form with data from the database."""
    customer = query_db('SELECT * FROM customers WHERE id = ?', [customer_id], one=True)
    
    if customer is None:
        return "Customer not found", 404
        
    return render_template('form.html', customer=customer)


@app.route('/static/faces/<filename>')
def send_face_image(filename):
    """Serves the cropped face images."""
    return send_from_directory(app.config['FACE_FOLDER'], filename)


if __name__ == '__main__':
    # Ensure static folders exist
    os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
    os.makedirs(app.config['FACE_FOLDER'], exist_ok=True)
    app.run(debug=True)
